In [1]:
import requests

In [2]:
import geopandas
import pandas as pd

## Get 2024 Election Data

In [3]:
election_data_url = "https://api-internal.sos.ca.gov/returns/maps/sorted/president"
r = requests.get(url=election_data_url)

In [4]:
election_data_2024 = r.json()

In [6]:
rep_votes_list = []
for county, data in election_data_2024.items():
    total_votes = 0
    for candidate in data["candidates"]:
        if candidate["Name"] == "Donald J. Trump":
            rep_votes = int(candidate["Votes"].replace(",", ""))
            total_votes += rep_votes
        else:
            total_votes += int(candidate["Votes"].replace(",", ""))

    rep_votes_list.append({"county": county, "votes": rep_votes, "totals": total_votes})

rep_results_2024_df = pd.DataFrame(dem_votes_list)

## Get 2020 Election Data

In [8]:
election_data_2020 = dfs = pd.read_excel(
    "data/pres-summary-by-county.xlsx", sheet_name=None
)

In [9]:
election_data_2020_df = election_data_2020["SOV Statewide Contest Details"]
election_data_2020_df = election_data_2020_df[
    (election_data_2020_df["Unnamed: 0"].isna() != True)
    & (election_data_2020_df["Unnamed: 0"] != "  Percent")
]

In [10]:
rename_dict = {}
for col in list(election_data_2020_df.columns):
    if col == "Unnamed: 0":
        new_col = "county"
    else:
        new_col = col.replace("\n", "")
    rename_dict[col] = new_col

election_data_2020_df = election_data_2020_df.rename(columns=rename_dict)
election_data_2020_df.loc[election_data_2020_df.county == "Butte1", "county"] = "Butte"

In [11]:
election_data_2020_df = election_data_2020_df.fillna(0)

/tmp/ipykernel_11775/3198050343.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  election_data_2020_df = election_data_2020_df.fillna(0)


In [15]:
counts_2020_df = election_data_2020_df.loc[
    :, election_data_2020_df.columns != "county"
].astype("int64")
counts_2020_df["totals"] = counts_2020_df.sum(axis=1, numeric_only=True)
counts_2020_df["county"] = election_data_2020_df["county"]
election_data_2020_df = counts_2020_df

In [18]:
election_data_2020_df = election_data_2020_df[["county", "Donald J. Trump*", "totals"]]
rep_results_2020_df = election_data_2020_df.rename(
    columns={"Donald J. Trump*": "votes"}
)
rep_results_2020_df["county"] = (
    rep_results_2020_df["county"].str.replace(" ", "-").str.lower()
)

## Combined Data

In [21]:
rep_results_2020_df["percent"] = (
    rep_results_2020_df["votes"] / rep_results_2020_df["totals"]
)
rep_results_2024_df["percent"] = (
    rep_results_2024_df["votes"] / rep_results_2024_df["totals"]
)

In [23]:
rep_results_2020_df["votes"] = rep_results_2020_df["votes"].astype(int)
rep_results_2024_df["votes"] = rep_results_2024_df["votes"].astype(int)
rep_vote_total = pd.merge(
    rep_results_2020_df,
    rep_results_2024_df,
    suffixes=("_2020", "_2024"),
    left_on="county",
    right_on="county",
    how="inner",
    validate="one_to_one",
)

In [24]:
rep_vote_total["vote_difference"] = (
    rep_vote_total["percent_2024"] - rep_vote_total["percent_2020"]
).astype(float)

## Get Geospatial data

In [26]:
geojson_data = "data/California_County_Boundaries_6550485670014028237.geojson"
california_counties = geopandas.read_file(geojson_data)

In [27]:
california_counties_minus_islands = california_counties[
    california_counties["ISLAND"] != "Channel Islands"
]

In [28]:
california_counties_minus_islands["COUNTY_NAME"] = (
    california_counties_minus_islands["COUNTY_NAME"].str.replace(" ", "-").str.lower()
)

/mnt/c/Users/Winston/Documents/GitHub/VineMapper/California_Republican_Pres_Turnout_2020_2024/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
california_counties_minus_islands = california_counties_minus_islands.rename(
    columns={"COUNTY_NAME": "county"}
)

## Merge data into Geojson for map making

In [32]:
california_counties_minus_islands = california_counties_minus_islands.merge(
    rep_vote_total, on="county", how="left"
)[["county", "geometry", "vote_difference"]]

In [33]:
california_counties_minus_islands.to_file("data/rep_voter_difference_2020_2024.gpkg")